In [1]:
import tensorflow as tf

# force gpu use
gpus = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(gpus[0], 'GPU')

2024-10-19 22:04:55.339855: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-19 22:04:55.378531: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-19 22:04:55.422953: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-19 22:04:55.436964: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-19 22:04:55.471935: I tensorflow/core/platform/cpu_feature_guar

In [2]:
import os

from pinn import PINNModel, model1, optm1
from data_gen import DataGenerator, real_u1, get_data

I0000 00:00:1729364698.959197   11279 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1729364698.959667   11279 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1729364698.960231   11279 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1729364699.065632   11279 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [3]:
session = 3
data = get_data(session)
EPOCHS = data['epochs']
koefs = data['big_range']
if not data["session_exists"]:
    os.makedirs(f"../models/s{session}")

True


In [4]:
model = PINNModel(
    model1(), optm1(), lossf="mae", initial_weights=DataGenerator.init_name(session))

# if there are no initial weights, save current as initial
if not os.path.exists(DataGenerator.init_name(session)):
    model.save(DataGenerator.init_name(session))

dg = DataGenerator(data['x'], data['y'], model.predict, real_u1)

In [5]:
area_grid_size = (50, 50)
border_grid_size = (50, 50)
dataset = dg.inner_pairs(area_grid_size), dg.border_pairs(border_grid_size)

for koef in koefs:
    if not os.path.exists(dg.name(koef, session)):
        model = PINNModel(
            model1(), optm1(), "mae", initial_weights=DataGenerator.init_name(session))
        # model.reset()
        model.fit(koef, *dataset, EPOCHS)
        model.save(dg.name(koef, session))

In [6]:
import shutil

shutil.make_archive(f"../models/s{session}", 'zip', f"../models/s{session}")

'/home/c0c0nut/Projects/PINN_Poisson/models/s3.zip'